In [23]:
import pandas as pd
import plotly.express as px
import numpy as np

In [24]:
PATH = '/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/data/'

In [25]:
sas = pd.read_csv(PATH + 'metropolitan-stop-and-search.csv')

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_48448/1198611584.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sas = pd.read_csv(PATH + 'metropolitan-stop-and-search.csv')


In [26]:
sas = sas.drop(['Part of a policing operation', 'Policing operation', 'Outcome linked to object of search', 'Removal of more than just outer clothing'], axis = 1)

In [20]:
# sas = sas[~(
#     (sas['Officer-defined ethnicity'].isin(['Other'])) &
#     (sas['Self-defined ethnicity'].isin([
#         'Other ethnic group - Any other ethnic group', 
#         'Not Stated (NS)', 
#         'Other ethnic group - Not stated', 
#         'Chinese or other ethnic group - Any other ethnic group (O9)',
#         'Mixed/Multiple ethnic groups - Any other Mixed/Multiple ethnic background',
#         'Mixed - Any other Mixed ethnic background (M9)',
#         'Chinese or other ethnic group - Any other ethnic group (O9)',
#     ]))
# )]

In [27]:
sas = sas[(sas['Officer-defined ethnicity'].isna() == False) & (sas['Self-defined ethnicity'].isna() == False)]

In [28]:
len(sas)

1606350

In [39]:
sas['Self-defined ethnicity'].unique()

array(['Mixed/Multiple ethnic groups - Any other Mixed/Multiple ethnic background',
       'Black/African/Caribbean/Black British - Any other Black/African/Caribbean background',
       'White - English/Welsh/Scottish/Northern Irish/British',
       'Black/African/Caribbean/Black British - Caribbean',
       'Other ethnic group - Not stated',
       'Black/African/Caribbean/Black British - African',
       'Asian/Asian British - Chinese',
       'Asian/Asian British - Bangladeshi',
       'Asian/Asian British - Any other Asian background',
       'White - Irish', 'White - Any other White background',
       'Asian/Asian British - Indian',
       'Other ethnic group - Any other ethnic group',
       'Mixed/Multiple ethnic groups - White and Black Caribbean',
       'Asian/Asian British - Pakistani',
       'Mixed/Multiple ethnic groups - White and Asian',
       'Mixed/Multiple ethnic groups - White and Black African',
       'Black or Black British - Any other Black ethnic background (

In [38]:
import re

In [52]:
string = ' asjfl;kasflk;ajflakfa '

' asjfl;kasflk;ajflakfa'

In [59]:

if re.match(r'^(Mixed)|(Other)|(Not)', 'Not Stated (NS)'):
    print('Mixed/unstated is found')

Mixed/unstated is found


In [63]:
# make a function to combine people into groups
def group_ethnicity(data):
    if data['Officer-defined ethnicity'] == 'Other':
        if re.match(r"^(White)", data['Officer-defined ethnicity']):
            return 'White'
        elif re.match(r"^(Black)", data['Officer-defined ethnicity']):
            return 'Black'
        elif re.match(r"^(Asian)|(Chinese)", data['Officer-defined ethnicity']):
            return 'Asian'
        elif re.match(r"^(Mixed)|(Other)|(Not)",data['Officer-defined ethnicity']):
            return 'Mixed'
    else:
        return data['Officer-defined ethnicity']
        

In [64]:
sas['generalized_ethnicity'] = sas.apply(group_ethnicity, axis = 1)

In [66]:
sas['generalized_ethnicity'].unique()

array(['Black', 'White', 'Asian', 'Mixed'], dtype=object)

In [16]:
sas['year'].dtype

NameError: name 'sas' is not defined

In [ ]:
sas['year'] = pd.to_datetime(sas['Date']).dt.year
sas['month'] = pd.to_datetime(sas['Date']).dt.month
sas['day'] = pd.to_datetime(sas['Date']).dt.day
sas['weekday'] = pd.to_datetime(sas['Date']).dt.day_name()

In [ ]:
px.histogram(sas[(sas['Gender'] == 'Female') & (sas['Age range'] == '18-24')], x = 'year', color = 'Officer-defined ethnicity')

In [ ]:
px.histogram(sas[(sas['Gender'] == 'Female') & (sas['Age range'] == '25-34')], x = 'year', color = 'Officer-defined ethnicity')

In [ ]:
px.histogram(sas[(sas['Gender'] == 'Female') & (sas['Age range'] == '10-17')], x = 'year', color = 'Officer-defined ethnicity')

In [ ]:
px.histogram(sas[(sas['Gender'] == 'Female') & (sas['Age range'] == 'over 34')], x = 'year', color = 'Officer-defined ethnicity')

In [ ]:
px.histogram(sas[(sas['Gender'] == 'Female') & (sas['Age range'] == 'under 10')], x = 'year', color = 'Officer-defined ethnicity')

In [ ]:
def create_heatmap_per_year(df):
    # Filter data for years 2011-2023
    years = range(2015, 2024)
    
    for year in years:
        # Filter data for the current year
        df_filtered = df[df['year'] == year].drop('year', axis = 1)

        # Create pivot table with counts for each combination of Crime type, Last outcome category, and Year
        pivot_table = df_filtered.pivot_table(index=['Legislation'], columns=['Object of search'], aggfunc=len, fill_value=0)

        # Create heatmap using Plotly
        fig = px.imshow(pivot_table.values,
                        labels=dict(x="Object of search", y="Legislation", color="Count"),
                        x=pivot_table.columns,
                        y=pivot_table.index,
                        color_continuous_scale='viridis')

        fig.update_layout(title=f'Crime Type and Last Outcome Category Heatmap for Year {year}',
                          width=800,  # Adjust width of the plot
                          height=600,  # Adjust height of the plot
                          margin=dict(l=50, r=50, t=100, b=100),  # Adjust margins
                          xaxis=dict(tickangle=-45),  # Rotate x-axis labels by -45 degrees
                          yaxis=dict(tickmode='array', tickvals=list(range(len(pivot_table.index))), ticktext=pivot_table.index))  # Disable aspect ratio constraint

        fig.show()

In [ ]:
create_heatmap_per_year(sas[['Object of search', 'Legislation', 'year']])

In [ ]:
sas

In [ ]:
sas["Object of search"].unique()

In [ ]:
px.histogram(sas, x = 'Age range', color = 'Officer-defined ethnicity')
# get the percentage

In [ ]:
sas_counts = sas.groupby('Self-defined ethnicity').count().sort_values(by = 'Type').reset_index()

In [ ]:
px.histogram(sas_counts, x= 'Self-defined ethnicity', y = 'Type')

In [ ]:
import re

In [ ]:
if re.match(r"^(White)|(Asian)|(Black)", sas[sas['Officer-defined ethnicity'] == 'Other'].iloc[1]['Self-defined ethnicity']):
   print(re.match(r"^(White)|(Asian)|(Black)", sas[sas['Officer-defined ethnicity'] == 'Other'].iloc[1]['Self-defined ethnicity'])[0])
else:
    print('penis')

In [ ]:
def change_ethnicity(ethnicity):
    if re.match(r"^(White)|(Asian)|(Black)", ethnicity):
        return re.match(r"^(White)|(Asian)|(Black)", ethnicity)[0]

In [ ]:
sas[sas['Officer-defined ethnicity'] == 'Other']['Officer-defined ethnicity'] = sas[sas['Officer-defined ethnicity'] == 'Other']['Self-defined ethnicity'].apply(change_ethnicity)

In [ ]:
sas[sas['Officer-defined ethnicity'] == 'Other']['Officer-defined ethnicity'] 

In [ ]:
# 31,497 occurences where officer-defined ethnicity is NaN
# 12,597 occurences from above do have the self defined ethnicity

In [ ]:
px.histogram(sas[(sas['Officer-defined ethnicity'].isna()) & (sas['Self-defined ethnicity'].isna() == False)].groupby('Self-defined ethnicity').count().sort_values(by = 'Type').reset_index(), x = 'Self-defined ethnicity', y = 'Type')

In [ ]:
# sas[(sas['Officer-defined ethnicity'].isna()) & (sas['Self-defined ethnicity'].isna() == False)].groupby('Self-defined ethnicity').count().sort_values(by = 'Type').reset_index()

In [ ]:
sas['year'] = pd.to_datetime(sas['Date']).dt.year

In [ ]:
sorted(sas['year'].unique())

In [ ]:
sas.columns

In [ ]:
pd.to_datetime(sas['Date']).dt.day.unique()

In [ ]:
px.histogram(sas, x= 'Officer-defined ethnicity', color = 'Self-defined ethnicity')

# Pick the ethnicity properly

In [ ]:
def change_ethnicity(ethnicity):
    if re.match(r"^(White)|(Asian)|(Black)", ethnicity):
        return re.match(r"^(White)|(Asian)|(Black)", ethnicity)[0]

In [ ]:
smth = sas[sas['Self-defined ethnicity'].isna() == False]
smth1 = smth[smth['Self-defined ethnicity'].str.startswith('White')]
smth2 = smth[smth['Self-defined ethnicity'].str.startswith('Black')]
smth3 = smth[smth['Self-defined ethnicity'].str.startswith('Asian')]

In [ ]:
count_total = smth1.groupby('Officer-defined ethnicity').count()[['Type']]
count_total['percentage'] = count_total['Type']/count_total['Type'].sum() * 100
count_total_white = count_total.reset_index().sort_values(by = 'percentage', ascending = False)
px.histogram(count_total_white, x = 'Officer-defined ethnicity', y = 'percentage')

In [ ]:
count_total = smth2.groupby('Officer-defined ethnicity').count()[['Type']]
count_total['percentage'] = count_total['Type']/count_total['Type'].sum() * 100
count_total_black = count_total.reset_index().sort_values(by = 'percentage', ascending = False)
px.histogram(count_total_black, x = 'Officer-defined ethnicity', y = 'percentage')

In [ ]:
px.histogram(smth2, x = 'Officer-defined ethnicity')

In [ ]:
count_total = smth3.groupby('Officer-defined ethnicity').count()[['Type']]
count_total['percentage'] = count_total['Type']/count_total['Type'].sum() * 100
count_total_asian = count_total.reset_index().sort_values(by = 'percentage', ascending = False)
px.histogram(count_total_asian, x = 'Officer-defined ethnicity', y = 'percentage')

In [ ]:
px.histogram(smth3, x = 'Officer-defined ethnicity')

## Locate the neighbourhood/borough

## Types per: Gender/Age/Ethnicity/Date

In [ ]:
px.histogram(sas, x = 'Type', color = 'Gender')

In [ ]:
px.histogram(sas, x = 'Type', color = 'Age range')

In [ ]:
px.histogram(sas, x = 'Type', color = 'year')

In [ ]:
skrr = sas.groupby(['Type', 'weekday']).count()[['month']].reset_index().sort_values(by = 'month')

In [ ]:
px.histogram(skrr, x = 'Type', y = 'month',color = 'weekday')

## Date per: Age/Ethnicity/Gender

In [ ]:
px.histogram(sas, x = 'weekday', color = 'Gender')

In [ ]:
px.histogram(sas, x = 'weekday', color = 'Age range')

In [ ]:
px.histogram(sas, x = 'weekday', color = 'Officer-defined ethnicity')

## Age per Gender

In [ ]:
px.histogram(sas, x = 'Age range', color = 'Gender')

In [ ]:
px.histogram(sas, x = 'Age range', color = 'Officer-defined ethnicity')

## Age per Ethnicity

## Gender per Ethnicity

In [ ]:
px.histogram(sas, x= 'Gender', color = 'Officer-defined ethnicity')

## Self defined ethnicity vs. Officer-defined ethnicity

### How good/bad do officers identify ethnicity

## Object of Search per Gender/Age/Ethnicity/Outcome/Date/Legislation

In [ ]:
pivot_table = sas[['Object of search', 'Gender']].pivot_table(index = 'Gender', columns = 'Object of search', aggfunc = len, fill_value = -1)
fig = px.imshow(pivot_table.values,
                        labels=dict(x="Object of Search", y="Gender", color="Count"),
                        x=pivot_table.columns,
                        y=pivot_table.index,
                        color_continuous_scale='viridis')

fig.show();

In [ ]:
pivot_table = sas[sas['Gender'] == 'Male'][['Object of search', 'Age range']].pivot_table(index = 'Age range', columns = 'Object of search', aggfunc = len, fill_value = -1)
fig = px.imshow(pivot_table.values,
                        labels=dict(x="Object of Search", y="Age range", color="Count"),
                        x=pivot_table.columns,
                        y=pivot_table.index,
                        color_continuous_scale='viridis')

fig.show();

In [ ]:
pivot_table = sas[sas['Gender'] == 'Female'][['Object of search', 'Age range']].pivot_table(index = 'Age range', columns = 'Object of search', aggfunc = len, fill_value = -1)
fig = px.imshow(pivot_table.values,
                        labels=dict(x="Object of Search", y="Age range", color="Count"),
                        x=pivot_table.columns,
                        y=pivot_table.index,
                        color_continuous_scale='viridis')

fig.show();

In [ ]:
pivot_table = sas[['Object of search', 'Outcome']].pivot_table(index = 'Object of search', columns = 'Outcome', aggfunc = len, fill_value = -1)
fig = px.imshow(pivot_table.values,
                        labels=dict(x="Outcome", y="Object of Search", color="Count"),
                        x=pivot_table.columns,
                        y=pivot_table.index,
                        color_continuous_scale='viridis')


# Manually set the range for x and y axes
fig.update_xaxes(tickmode='array', tickvals=list(range(len(pivot_table.columns))), ticktext=pivot_table.columns)
fig.update_yaxes(tickmode='array', tickvals=list(range(len(pivot_table.index))), ticktext=pivot_table.index)
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()

## Outcomes per Gender/Age

In [ ]:
px.histogram(sas, x = 'Outcome', color = 'Gender')

In [ ]:
px.histogram(sas, x = 'Outcome', color = 'Age range')

In [ ]:
import pandas as pd

In [ ]:
pas = pd.read_csv('/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/data/PAS_borough')

In [ ]:
pas

In [ ]:
pas['Measure'].unique()

In [ ]:
pas[pas['Measure'] == 'Trust MPS']

In [ ]:
pas[(pas['Date'] == '2014-12-31') & (pas['Borough'] == 'Barking and Dagenham')]

In [ ]:
# Create a pivot table
pivot_table = pas.pivot_table(index=['Date', 'Borough'], columns='Measure', values='Proportion', fill_value=None)

# Reset index to make it columns again
pivot_table.reset_index(inplace=True)

pivot_table.to_json('/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/data/PAS_borough_new.csv', index = False)